
<center>
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> Tarea N°1</h1>
    <h2>Ethiel Carmona, 201773533-3</h2>
</center>

<p>

</p>

## Instrucciones

* La tarea es individual.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 30 de Abril a las **18:00 hrs**.  Se aceptarán entregas hasta las 19:00 hrs sin descuento en caso de existir algun problema, posteriormente existirá un descuento lineal hasta las 20:00 hrs del mismo día.
* Las tareas que sean entregadas antes del jueves a mediodía recibirán una bonificación de 10 puntos
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).




## Introducción 

En esta primera tarea de INF/ILI-285, versión 2020-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Significancia, Errores de Cancelación y Búsqueda de Ceros. El desarrollo de cada uno de esos temas se presenta en una serie de preguntas, donde ustedes deben ir decidiendo, pregunta a pregunta, como cada uno de los temas se aplica. En general, los temas no se analizan por separado, sino de manera acoplada. Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.

In [3]:
import numpy as np
import scipy.special as sp

## Problemas

### 1. Simulador (50 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=x^3 - A\,x^2 + A\,x - 1
\end{equation}


Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$) y la cantidad de bits que tiene en la mantisa un computador ficticio, el manejo del exponente debe considerarse de las misma forma que lo maneja _double precision_.
Esta implementación debe calcular las raíces de $f$ sin perdida de significancia con la cantidad de bits disponibles para la mantisa.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

Considere que en ese computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces.  Considere el siguiente ejemplo:

```python
# Alg. Base
a = 9819824.624837
b = 148736.523476
c = a+b

# Alg. con Representación de Punto Flotante de 'bits_mant' bits en la mantisa.
am = f_new_rep(9819824.624837,bits_mant) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant) # Aproximar el input en la nueva representación.
cm = f_new_rep(am+bm, bits_mant) # Aproximar el output de la suma y cada operación en la nueva representación.
```


In [4]:
# Funcion: parteEntera
# Parametros:
# - entero: Numero de tipo int
# Descripcion:
# - Transforma la parte entera de un numero real a binario
# Retorno:
# - lista: Lista con el valor en binario
def parteEntera(entero):
    lista = []
    valor = entero
    flag = True
    while flag:
        if int(valor / 2) == 0:
            flag = False
        lista.insert(0, str(valor % 2))
        valor = int(valor / 2)
    return lista

In [5]:
# Funcion: parteDecimal
# Parametros:
# - decimal: Numero de tipo float
# Descripcion:
# - Transforma la parte decimal de un numero real a binario
# Retorno:
# - lista: Lista con el valor en binario
def parteDecimal(decimal):
    lista = []
    aparecidos = []
    valor = decimal
    flag = True
    if decimal == 0:
        lista = ["0"]
        return lista
    while flag:
        valor = valor * 2
        if valor >= 1:
            resto = valor - 1
            if resto in aparecidos:
                flag = False
            lista.append("1")
            aparecidos.append(resto)
            valor = resto
        else:
            if valor in aparecidos:
                flag = False
            lista.append("0")
            aparecidos.append(valor)
    return lista

In [6]:
# Funcion: correrFraccion
# Parametros:
# - list_frac: Lista que contiene la parte fraccionaria en binario del numero real
# Descripcion:
# - Calcula la cantidad de posiciones que se debe mover el punto que separa la parte entera de la fraccionaria binaria
# Retorno:
# - contador: Numero de posiciones a moverse (De cierta forma, es el exponente)
def correrFraccion(list_frac):
    largo = len(list_frac)
    contador = 0
    while contador < largo:
        if list_frac[contador] == "1":
            break
        else:
            contador += 1
    return contador

In [7]:
# Funcion: correrEntera
# Parametros:
# - list_entera: Lista que contiene la parte entera en binario del numero real
# Descripcion:
# - Calcula la cantidad de posiciones que se debe mover el punto que separa la parte entera de la fraccionaria binaria
# Retorno:
# - Retorna el largo de la parte binaria entera menos 1, asi podemos pasar de 10101 a 1.0101, básicamente.
def correrEntera(list_entera):
    return len(list_entera) - 1

In [8]:
# Funcion: suma
# Parametros:
# - lista: Realiza la suma de la mantisa con 1, para poder respetar las condiciones de aproximacion del bit 'bit_mant + 1'
# Descripcion:
# - Obtiene la mantisa redondeada
# Retorno:
# - lista: Lista con la mantisa redondeada
def suma(lista):
    largo = len(lista) - 1
    while largo > 0:
        if lista[largo] == '0':
            lista[largo] = '1'
            break
        else:
            lista[largo] == '0'
            largo -= 1
    return lista

In [9]:
# Funcion: suma_mantisa
# Parametros:
# - lista: Lista que contiene la mantisa en bits
# Descripcion:
# - Calcula si se debe sumar un 1 a la mantisa nueva
# Retorno:
# - lista: Lista con la mantisa redondeada
def suma_mantisa(lista, mantisa):
    if lista[mantisa] == '0':
        #Si bit 'mantisa+1' es 0
        return lista[:mantisa]
    else:
        #Bit 'mantisa+1' es 1
        iteracion = lista[mantisa:]
        hay_bit_1 = 0
        largo_iteracion = len(iteracion)
        contador = 1
        while contador < largo_iteracion:
            if iteracion[contador] == '1':
                hay_bit_1 = 1
                break
            contador += 1
        lista = lista[:mantisa]
        if hay_bit_1 == 1:
            #SUMAR
            return suma(lista)
        else:
            if lista[mantisa - 1] == '1':
                #SUMAR
                return suma(lista)
            else:
                return lista

In [10]:
# Funcion: exponenciacion
# Parametros:
# - entero: lista que con los bits de la parte entera 
# - fraccion: lista con los bits de la parte fraccionaria
# Descripcion:
# - Transforma de binario a base decimal
# Retorno:
# - flotante con la suma previamente mencionada
def exponenciacion(entero, fraccion):
    largo_entero = len(entero)
    largo_frac = len(fraccion)
    val_entero = 0
    val_frac = 0.0
    posicion_entero = 0
    while posicion_entero < largo_entero:
        val_entero += int(entero[posicion_entero]) * 2**(largo_entero - 1 - posicion_entero)
        posicion_entero += 1
    posicion_frac = 0
    potencia = -1
    while posicion_frac < largo_frac:
        val_frac += int(fraccion[posicion_frac]) * 2**(potencia)
        potencia -= 1
        posicion_frac += 1
    return val_entero + val_frac

In [11]:
# Funcion: binarioDecimal
# Parametros:
# - lista: Lista que contiene la mantisa
# - exponente: Cantidad de posiciones que se debe correr el punto
# Descripcion:
# - Prepara la mantisa ajustandola para dejarla en binario con una parte entera y una parte fraccionaria
# Retorno:
# - flotante con la suma previamente mencionada
def binarioDecimal(lista, exponente):
    x = abs(len(lista) - abs(exponente))
    aux = ['0']
    if x != 0:
        lista += aux * x
    entera = []
    fraccion = []
    if exponente >= 0:
        entera = lista[:exponente]
        entera.insert(0, '1')
        fraccion = lista[exponente:]
    else:
        fraccion = ['0'] * (-exponente - 1)
        entera = ['0']
        lista.insert(0, '1')
        fraccion = fraccion + lista
    return exponenciacion(entera, fraccion)
        

In [12]:
"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
output
eval      : (double) resultado obtenido
"""
# Algoritmo de representación de punto flotante modificada.
def f_new_rep(x, bits_mant):
    if x == 0:
        return 0
    signo = 0 if x >= 0 else 1
    x = abs(float(x))
    x = format(x, '.52f')
    x_string = str(x)
    
    pos_dot = x_string.find('.')
    x_entero = x_string[0:pos_dot]
    x_decimal = x_string[pos_dot:]
    
    p_entera_bin = parteEntera(int(x_entero))
    p_decimal_bin = parteDecimal(float(x_decimal))
    
    corrimiento = correrFraccion(p_decimal_bin) + 1 if len(p_entera_bin) == 1 and p_entera_bin[0] == '0' else -correrEntera(p_entera_bin)
    exponente = -corrimiento
    
    if corrimiento >= 0: #muevo a la derecha
        p_decimal_bin = p_decimal_bin[corrimiento:]
    else: #muevo a la izquierda
        p_decimal_bin = p_decimal_bin
        p_decimal_bin = p_entera_bin[len(p_entera_bin) + corrimiento:] + p_decimal_bin
        #p_entera_bin = p_entera_bin[:len(p_entera_bin) + corrimiento]
        
    p_decimal_bin += ['0'] * bits_mant
    
    p_decimal_bin = suma_mantisa(p_decimal_bin, bits_mant)
    result = binarioDecimal(p_decimal_bin, exponente) * (-1)**signo
    return  result

De la ecuación:

\begin{equation}
f(x)=x^3 - A\,x^2 + A\,x - 1
\end{equation}

Se puede observar que $x = 1$ es una raíz. Luego, con esta solución y al hacer división de polinomios para encontrar las raíces faltantes, obtenemos:

\begin{equation}
f(x) = (x-1)(x^2 + (1-A)x + 1)
\end{equation}

Donde podemos ocupar la fórmula de raíces para ecuaciones cuadráticas:

\begin{equation}
x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
\end{equation}

Obteniendo las siguientes soluciones:

\begin{equation}
x_{1} = \frac{-b + \sqrt{b^2 - 4ac}}{2a} = \frac{A - 1 + \sqrt{A^2 - 2A - 3}}{2}
\end{equation}

\begin{equation}
x_{2} = \frac{-b - \sqrt{b^2 - 4ac}}{2a} = \frac{A - 1 - \sqrt{A^2 - 2A - 3}}{2}
\end{equation}

Para evitar perdida de significancia, la solucion $x_{1}$ se multiplica por un uno conveniente:

\begin{equation}
x_{1} = \frac{-b + \sqrt{b^2 - 4ac}}{2a} \frac{-b - \sqrt{b^2 - 4ac}}{-b - \sqrt{b^2 - 4ac}} = \frac{2c}{- b - \sqrt{b^2 - 4ac}}
\end{equation}

Obteniendo así las siguientes soluciones:

\begin{equation}
x_{1} = \frac{2}{A - 1 - \sqrt{A^2 - 2A - 3}}
\end{equation}

\begin{equation}
x_{2} = \frac{A - 1 - \sqrt{A^2 - 2A - 3}}{2}
\end{equation}

\begin{equation}
x_{3} = 1
\end{equation}

In [13]:
def f_find_roots(A, bits_mant):
    A = f_new_rep(A, bits_mant)
    eval_1 = f_new_rep(A-1, bits_mant) #A - 1
    eval_2 = f_new_rep(A * A, bits_mant) #A^2
    eval_3 = f_new_rep(2 * A, bits_mant) #2A
    eval_4 = f_new_rep(eval_2 - eval_3, bits_mant) #A^2 - 2A
    eval_5 = f_new_rep(eval_4 - 3, bits_mant) #A^2 - 2A - 3
    eval_6 = f_new_rep(np.sqrt(eval_5), bits_mant) # RAIZ()
    eval_7 = f_new_rep(eval_1 + eval_6, bits_mant) # A - 1 + RAIZ()
    sol_1 = f_new_rep(2 / eval_7, bits_mant)
    sol_2 = f_new_rep(eval_7 / 2, bits_mant)
    x_roots = [1, sol_1, sol_2]
    return x_roots

### Polinomios de Legendre (50 puntos)
Dada la función compuesta $f$
\begin{equation}
f_{n,m}(x) = L_n(C_m(x)),
\end{equation}
donde $L_n$ es conocido como el polinomio de Legendre de grado $n$ definido de la siguiente forma:
\begin{equation}
L_{n}(x)=\frac{1}{2^{n}} \sum_{k=0}^{n}\left(\begin{array}{l}
n \\
k
\end{array}\right)^{2}(x-1)^{n-k}(x+1)^{k},
\end{equation}
y $C_m$ es el polinomio de Chebyshev
\begin{equation}
C_m(x) = \cos(m \cdot \arccos(x)).
\end{equation}

Utilizando el método de Bisección y Punto fijo se pide obtener la raíz de $f$ más cercana a $0.5$ dado un valor de $m$ y $n$



*Hint: Las raíces de Legendre son conocidas*



-

-

Para la función Bisección, definí un intervalo [a, b] cercano a 0,5 como se puede observar dentro de la misma función.
Este intervalo lo definí en funcion de $m$ ya que a medida que este aumenta, la frecuencia del coseno también lo hace, por lo que las posibles soluciones de este para Legendre estarán cada vez mas cercanas.
El factor $m$ dentro de este mismo, me brinda de alguna forma "proporcional", la precisión necesaria para poder adentrarme lo suficiente en el intervalo que tenga la raíz mas cercana a 0,5, evitando así retornar alguna que esté más alejada.

In [14]:
c_m = lambda m, x: np.cos(m * np.arccos(x))

In [15]:
# Funcion: l_n
# Parametros:
# - x: valor obtenido de c_m (del coseno)
# - n: Polinomio n-ésimo de Legendre
# Descripcion:
# - Evalúa el resultado x en el polinomio
# Retorno:
# - solución flotante para Legendre con el valor x
def l_n(x, n):
    p = sp.legendre(n)
    return p(x)

In [16]:
"""
input
n: (int) grado del polinomio de Legendre
m: (int) grado del polinomio de Chebyshev
tol: (double) tolerancia para la detención del algoritmo
output
root: (double) raiz obtenida
"""
def f_Biseccion(n, m, tol):
    ##
    a = 0.5 - (1/m)
    b = 0.5 + (1/m)
    while (b - a) / 2 > tol:
        c = (a + b) / 2
        if l_n(c_m(m, c), n) == 0:
            break
        if l_n(c_m(m, a), n) * l_n(c_m(m, c), n) < 0:
            b = c
        else:
            a = c
    return (a + b) / 2

Para el caso de FPI, considerar (a + b)/2 como x_0 no es la mejor idea, ya que esta iterará hasta encontrar la primera raíz, la cual, dependiendo del valor de $m$ no necesariamente sería la mas cercana. Por lo cual, tomando el hint de que las raíces de Legendre son conocidas, busqué la raíz mas cercana a 0.5 (no evaluada en el coseno) y la utilicé como valor inicial.

In [18]:

def raizCercana(n):
    raices = sp.roots_legendre(n)[0]
    aux = 500 #Numero lo suficientemente grande
    for raiz in raices:
        if abs(raiz - 0.5) < abs(aux - 0.5):
            aux = raiz
    return aux

"""
input
n: (int) grado del polinomio de Legendre
m: (int) grado del polinomio de Chebyshev
tol: (double) tolerancia para la detención del algoritmo
output
root: (double) raiz obtenida
"""
def f_FPI(n, m, tol):
    aux = raizCercana(n)
    h = lambda x: x - aux / 10**3 + c_m(m, x) / 10**3
    
    raiz = h(aux)
    flag = True
    while flag:
        x_0 = h(raiz)
        if abs(x_0 - raiz) < tol:
            break
        raiz = x_0
    return x_0

###  Determinantes (20 puntos)

Dada una matriz de dimensiones $ n \times n$ de la forma:
\begin{equation}
A
=
\begin{pmatrix}
a_{1,1} & a_{1,2}  & \dots & a_{1,n} \\
a_{2,1} & a_{2,2}  & \dots & a_{2,n} \\
\vdots &  \vdots   & \ddots & \vdots \\
a_{n,1} & a_{n,2}  & \dots & a_{n,n}
\end{pmatrix}
=
\begin{pmatrix}
\mathbf{r}_1 \\
\mathbf{r}_2 \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
donde $\mathbf{r}_k$ representa la $k$-ésima fila de la matriz $A$.
Considere la siguiente matriz $C_{i,j}(\alpha)$,
\begin{equation}
C_{i,j}(\alpha)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_i\,(1-\alpha)+\mathbf{r}_j\,\alpha \\
\vdots \\
\mathbf{r}_j\,(1-\alpha)+\mathbf{r}_i\,\alpha \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
de lo cual sabemos que \begin{align}$C_{i,j}(0)=A$ y que $C_{i,j}(1)$\end{align} es la matriz $A$ donde se intercambiaron las filas $i$ y $j$, es decir:
\begin{equation}
C_{i,j}(1)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_j \\
\vdots \\
\mathbf{r}_i \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix}.
\end{equation}
De las relaciones anteriores podemos concluir que el determinante de la matriz $A$, denominado $D=\det(A)$, es igual al determinante de $C_{i,j}(0)$, es decir $\det(C_{i,j}(0))=\det(A)=D$.
Por el otro lado, el determinante de $C_{i,j}(1)$ es $-D$, dado que es el intercambio de las filas $i$ y $j$ de la matriz $A$.
Por lo cual podemos concluir que $-D\leq \det(C_{i,j}(\alpha))\leq D$.

Utilizando el método de Bisección debe encontrar el valor de $\alpha$ con $p$ decimales de precisión que permitan que, dado los índices de las filas $i$, $j$, y $i\neq j$, el determinante de la matriz sea igual a $d$, donde $d\in[-D,D]$. 

Para esto se debe implementar una función que reciba la matriz $A$, las filas $i$ y $j$, y $p$; y retorne $\widehat{\alpha}$ tal que $\det(C_{i,j}(\widehat{\alpha}))=d$.

En general lo que hice en esta pregunta fue comparar los determinantes.
Lo que si debo destacar fue la obtención de la cantidad de iteraciones, la cual fue usando:

\begin{equation}
[1]: 0,5 x 10^{-p} = \frac{|b - a|}{2^{n+1}}
\end{equation}

Despejando $n$ (cantidad maxima de iteraciones), tenemos:

\begin{equation}
n = log_{2}{10^p}
\end{equation}

In [19]:
"""
input
A: (array n x n) matriz
i: (int) índice de la fila "i".
j: (int) índice de la fila "j".
p: (int) cantidad de decimales de precision 
d: (double) valor requerido del determinante de $C_{i,j}(\alpha)$, i.e. $\det(C_{i,j}(\widehat{\alpha}))=d$.
output
alpha_hat: (double) alpha_hat tal que det(C_{i,j}(alpha_hat))=d.
"""
det = lambda x: np.linalg.det(x)

def determinante(A, i, j, c):
    B = np.copy(A)
    B[i] = np.copy(A)[i] * (1 - c) + np.copy(A)[j] * c
    B[j] = np.copy(A)[j] * (1 - c) + np.copy(A)[i] * c
    return det(B)

def find_alpha_hat(A,i,j,p,d):
    h = lambda x: d - x
    a = 0.
    b = 1.
    n = round(np.log2(10**p))
    contador = 0
    #Casos borde
    if abs(d) > abs(det(A)):
        if d*det(A) < 0:
            return 1
        else:
            return 0
    #Casos normalitos
    while contador < n:
        c = (a + b) / 2
        det_a = determinante(A, i, j, a)
        det_b = determinante(A, i, j, b)
        det_c = determinante(A, i, j, c)
        h_a = h(det_a)
        h_b = h(det_b)
        h_c = h(det_c)
        if h_c == 0:
            break
        if h_a * h_c < 0:
            b = c
        else:
            a = c
        contador += 1
    
    return round(((a + b) / 2), p)

# Referencias

Si corresponde

[1] Sección 1.1.2, Numerical Analysis 2nd Edition, Timothy Sauer.

*A modo general, el comportamiento de las bisecciones o fpi se comportan similar a las enseñadas por los profesores, estas se pueden encontrar dentro de sus Jupyter. En particular, me guíe por:

Paralelo 200, Jupyter-Notebook, INF-285/material/03_raices_1D/ejemplo_biseccion_ipf.ipynb